In [1]:
import pandas as pd
from torch import nn
from torch.optim import SGD
from pandas_ml_common.utils.column_lagging_utils import lag_columns

from pandas_ml_utils import FeaturesAndLabels, RegressionSummary
from pandas_ml_utils_torch import PytorchNN, PytorchModel
from pandas_ml_utils_torch.merging_cross_folds import take_the_best


## Fist lets load some data

In [2]:
df = pd.read_csv('SPY.csv')
df.tail()


,Date,Open,High,Low,Close,Adj Close,Volume
6758,2019-11-29,314.859985,315.130005,314.059998,314.309998,314.309998,36592700
6759,2019-12-02,314.589996,314.660004,311.170013,311.640015,311.640015,75767800
6760,2019-12-03,308.649994,309.640015,307.130005,309.549988,309.549988,73941700
6761,2019-12-04,310.700012,312.119995,310.320007,311.459991,311.459991,49080000
6762,2019-12-05,312.230011,312.250000,310.580109,312.089996,312.089996,40781669


## Fit Regularized Model
Now we just build a very naive model where we add an L1 and L2 penalization term. Don't expect the model to predict
anything useful

In [3]:
with df.model() as m:

    class Net(PytorchNN):

        def __init__(self):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(10, 4),
                nn.Tanh(),
                nn.Linear(4, 4),
                nn.Tanh(),
                nn.Linear(4, 1),
                nn.Tanh(),
            )

        def L1(self):
            # path to the parameters which should be regularized
            # the path is constructed from self.named_parameters() and allows the use of wildcards
            return {'net/0/**/weight': 0.02}

        def L2(self):
            return {
                'net/0/**/weight': 0.02,
                'net/2/**/weight': 0.05
            }

        def forward_training(self, x):
            return self.net(x)

    fit = m.fit(
        PytorchModel(
            FeaturesAndLabels(
                [lambda df: lag_columns(df["Close"].pct_change(), range(10))],
                [lambda df: df["Close"].pct_change().shift(-1)]),
            Net,
            nn.MSELoss,
            lambda params: SGD(params, lr=0.01, momentum=0.0),
            merge_cross_folds=take_the_best,
            summary_provider=RegressionSummary
        ),
        epochs=100,
        verbose=1
    )

fit




epoch: 0, train loss: 0.6320877075195312, test loss: 0.6317884922027588
epoch: 1, train loss: 0.6193536520004272, test loss: 0.6190574169158936
epoch: 2, train loss: 0.6066455841064453, test loss: 0.6063526272773743
epoch: 3, train loss: 0.5939859747886658, test loss: 0.5936960577964783
epoch: 4, train loss: 0.581396758556366, test loss: 0.5811101198196411
epoch: 5, train loss: 0.5689002871513367, test loss: 0.568617045879364
epoch: 6, train loss: 0.5565187931060791, test loss: 0.5562388896942139
epoch: 7, train loss: 0.5442742109298706, test loss: 0.5439976453781128
epoch: 8, train loss: 0.5321876406669617, test loss: 0.5319145917892456
epoch: 9, train loss: 0.5202797651290894, test loss: 0.5200100541114807
epoch: 10, train loss: 0.5085701942443848, test loss: 0.5083039402961731
epoch: 11, train loss: 0.49707743525505066, test loss: 0.4968145787715912
epoch: 12, train loss: 0.4858187437057495, test loss: 0.4855594038963318
epoch: 13, train loss: 0.474810391664505, test loss: 0.4745544

mean_gamma_deviance failed: (ValueError('Mean Tweedie deviance error with power=2 can only be used on strictly positive y and y_pred.'), 'error while calling <function mean_gamma_deviance at 0x7fdb7e4e0170>(FullArgSpec(args=[], varargs=\'args\', varkw=\'kwargs\', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={}))\n@_deprecate_positional_args\ndef mean_gamma_deviance(y_true, y_pred, *, sample_weight=None):\n    """Mean Gamma deviance regression loss.\n\n    Gamma deviance is equivalent to the Tweedie deviance with\n    the power parameter `power=2`. It is invariant to scaling of\n    the target variable, and measures relative errors.\n\n    Read more in the :ref:`User Guide <mean_tweedie_deviance>`.\n\n    Parameters\n    ----------\n    y_true : array-like of shape (n_samples,)\n        Ground truth (correct) target values. Requires y_true > 0.\n\n    y_pred : array-like of shape (n_samples,)\n        Estimated target values. Requires y_pred > 0.\n\n    sample_weight :